#   Big Data
## Algorithms: Sorting, Recursion and Data Structures
## Victor P. Debattista March 2017


We are going to start with a very simple exercise in recursion just to get used to it, then implement a couple of sorting algorithms, one O(n$^2$) and one O(n log n)

In [2]:
import numpy as np
import math
import random
import time

Write a function that computes the n$^{th}$ Fibonacci number if $(F_0,F_1) = (1,1)$.  By definition a Fibonacci number is one such that $F_n = F_{n-1} + F_{n-2}$. You should use recursion to solve this problem, not a loop.
Print the first 10 Fibonacci numbers.

In [2]:
def Fibo(n):
    if( n == 1 or n == 0 ):
        return 1   # can change these values to change the Fibonacci sequence
    else:
        return Fibo(n-1) + Fibo(n-2)

In [3]:
for i in range(0,10):
    print(Fibo(i))

1
1
2
3
5
8
13
21
34
55


Let us start exploring sorting. In the first step we want to create a list of N numbers which we will use as our list for sorting

In [10]:
random.seed(22)
N = 10000
#N = 10
data = []
for i in range(N):
    data.append(random.uniform(0.,10000.))

Our first sorting algorithm is the insertion sort.  How would you sort the list data into another list, data2, using the insertion sort?  (We want to use a second list so we preserve our original list.  Note that swapping in Python is done easily via tuples:
(A,B) = (B,A)
with no need for temporary variables.)  Calculate how long this took to run.

In [11]:
start_time = time.time()
data2 = []
data2.append(data[0])   # set first element equal; cannot set lists equal or data will be sorted
for i in range(1,N):
    data2.append(data[i])   # insert element by element
    j = i
    while( data2[j] < data2[j-1] ) and ( j > 0 ):
        (data2[j],data2[j-1]) = (data2[j-1],data2[j])  #tuple swapping
        j = j - 1  # update the current location of the new entry
end_time = time.time()
print("RAW[0:7]:",data[0:7])
print("SORTED[0:7]:",data2[0:7])
print("Execution time =",end_time-start_time,'seconds')

RAW[0:7]: [9582.093798172727, 1403.685900763948, 236.1614713882554, 9986.306536729146, 1842.5364570285308, 1205.9206321532502, 6514.212405579194]
SORTED[0:7]: [0.13393762374525053, 0.3062384190777312, 1.26708702534728, 2.961197729346443, 4.908756848494011, 5.125039670373921, 5.283946623794167]
Execution time = 16.692615270614624 seconds


In the next part we will develop the functionality of a heap.  Write a recursive function that, given a list arr, sifts up element k ensuring that a heap structure is obtained.  The function should return the list back.  Be careful with index of the parent, it must work for both daughter nodes.

In [4]:
def sift_up(arr, k):
# in python, the parent node of element k is (k-1)//2, assuming the root is k=0
    if( arr[k] > arr[(k-1)//2] ) and ( k > 0 ):
        (arr[k],arr[(k-1)//2]) = (arr[(k-1)//2],arr[k])  # tuple swap
        arr = sift_up( arr, (k-1)//2 )  # recursively sift_up on the next level up
    return arr

Next write a function that, given a list arr, filled to element k, inserts (pushes) a new element N to it, preserving the heap structure.  So this will need to use your sift function from above.

In [5]:
def push( arr, k, N ):
    arr.append(N)  
    k = k + 1 # increase size of heap
    arr = sift_up(arr,k)
    return arr,k

With these two functions, given a list of numbers, turn it into a heap by building a function heapify.  This should work by pushing every element of a list onto the heap.

In [6]:
def heapify( data ):
    heap = []
    l = -1
    for val in data:
        heap,l = push(heap,l,val)
    return heap,l

Now write a function to sift down for when we pop the maximum value off.  This is a recursive function.  Be careful about having 0 or 1 daughters and that sifting down always involve a swap with the larger of the two daughters if 2 exist.

In [7]:
def sift_down( heap, k, l ):    
# in python the daughter nodes are 2i + 1 and 2i + 2
# k is the node to sift down, l is the last leaf

    if( 2*k + 1 <= l ):   # ignore nodes without daughters, i.e. leaves
        if( 2*k + 2 > l ):  # this only has one daughter; check it 
            if( heap[k] < heap[2*k+1] ):
                (heap[k],heap[2*k+1]) = (heap[2*k+1],heap[k])  # tuple swap
                # cannot sift down further
        else:  # this is a node with 2 daughters; find the larger one first
            left = ( heap[2*k+1] > heap[2*k+2] )
            m = 2
            if( left ):
                m = 1  # offset for larger daughter
            if( heap[k] < heap[2*k+m] ):
                (heap[k],heap[2*k+m]) = (heap[2*k+m],heap[k])  # tuple swap
                heap = sift_down( heap, 2*k+m, l )  # recurse the sifting down
    return heap
            

Now add a function to pop the maximum value.  Remember that when popping the root, it will be replaced by the furthest leaf, which is then sifted down.  Return both the heap and its size.

In [8]:
def pop_heap( heap, l ):
# l is the last node in the heap; in our pop we move the root there, so it is in sorted order
    (heap[l],heap[0]) = (heap[0],heap[l])   
    l = l - 1 
    heap = sift_down(heap,0,l)
    return heap,l

We have all the building blocks in place for a heap sort, so let's write that next.  We start by creating the heap, then repeatedly popping the heap, placing the popped items at the tail of the list that holds the heap.  I.e. we are going to sort the list in place, needing no extra storage.

In [9]:
# we start by looking at a few entries in the array to sort
print('RAW:',data[0:7])

start_time = time.time()
# build the initial heap
heap,l = heapify(data)

# for fun show the first few nodes to verify this is a heap
print('HEAPED:',heap[0],'\n',heap[1],heap[2],'\n',heap[3],heap[4],heap[5],heap[6])

# now we pop the heap, placing the popped item at the end of the heap in sorted order
# %% time
for i in range(N-1,0,-1):
    heap,l = pop_heap(heap,i)
end_time = time.time()
    
# heap should now be a sorted array.  let's verify
print('SORTED:',heap[0:7])
print('DATA2:',data2[0:7])
print('Execution time =',end_time-start_time,'seconds')

RAW: [981, 929, 143, 248, 24, 627, 457]
HEAPED: 999 
 997 998 
 994 996 988 992
SORTED: [0, 1, 2, 3, 4, 5, 6]


NameError: name 'data2' is not defined

Let's now do a bin sort.  Now we're going to be a bit looser with this and directly use some of Python's sorting methods.  Start by writing a simple function that, given a value which is within a given range [lo,hi], finds the bin to place the element into if there are N bins.  If the value is out of range some flag value should be returned.

In [13]:
def bin_index(val,lo,hi,N):
# start with the sanity check that val satisfies lo <= val <= hi, else have a flag
    if( (val < lo) or (val > hi) ):
        tmp = -1
    else: 
        tmp = (val - lo) * N/(hi-lo)
    a = int(tmp)
    return a

In this final step we do a bin sort.  This is a bit more complicated.  The way to do this is to have a list of lists.  You can directly use Python's .sort method for lists, since this is purely illustrative.  Or you can use your heapsort from before if your code was general enough.

In [13]:
# let's start by testing out Python's sort directly
data3 = data.copy()
start_time = time.time()
data3.sort()
end_time = time.time()    
print("Raw:",data[0:7])
print("Sorted:",data3[0:7])    
print('Execution time =',end_time-start_time,'seconds')

# binsort
data4 = data.copy()
start_time = time.time()

bins = 100 

# set up empty list of buckets
mlist = []
for i in range(0,bins):
    mlist.append([])

# populate the buckets    
for val in data:
    i = bin_index(val,0.,10000.,bins)
    if( i >= 0 ):   # ignore values that were out of range
        mlist[i].append(val)
    
# sort the buckets, checking if they are empty first
for i in range(0,bins):
    if(len(mlist[i]) > 1):   # don't bother sorting bins with 1 entry
        mlist[i].sort()      # using Python's resident list sort for this

sorted_list = [c for v in mlist for c in v] # list comprehension to flatten the list of lists
end_time = time.time()   
print(sorted_list[0:7])

print('Execution time =',end_time-start_time,'seconds')

Raw: [9582.093798172727, 1403.685900763948, 236.1614713882554, 9986.306536729146, 1842.5364570285308, 1205.9206321532502, 6514.212405579194]
Sorted: [0.13393762374525053, 0.3062384190777312, 1.26708702534728, 2.961197729346443, 4.908756848494011, 5.125039670373921, 5.283946623794167]
Execution time = 0.026282072067260742 seconds
[0.13393762374525053, 0.3062384190777312, 1.26708702534728, 2.961197729346443, 4.908756848494011, 5.125039670373921, 5.283946623794167]
Execution time = 0.013864994049072266 seconds


Consider the quicksort.  Suppose it is given a sorted list.  If the pivot is always the leftmost value, what happens?  Suggest a solution.  You do not need to code this up.

If the list is sorted and the pivot is the leftmost element of the list, then the list is broken into an empty list and the full list.  Quicksort then recurses on this and is stuck in an infinite loop doing no work.
A simple solution is to move the pivot to the empty list if the other list is not empty.